In [1]:
library(fpp3)
library(stringr)
library(fastDummies)
library(future)
# library(forecast)
options(repr.plot.width=20, repr.plot.height=5)

── Attaching packages ──────────────────────────────────────────── fpp3 0.4.0 ──

✔ tibble      3.1.7     ✔ tsibble     1.1.3
✔ dplyr       1.0.9     ✔ tsibbledata 0.4.1
✔ tidyr       1.2.0     ✔ feasts      0.3.0
✔ lubridate   1.8.0     ✔ fable       0.3.2
✔ ggplot2     3.4.0     

── Conflicts ───────────────────────────────────────────────── fpp3_conflicts ──
✖ lubridate::date()    masks base::date()
✖ dplyr::filter()      masks stats::filter()
✖ tsibble::intersect() masks base::intersect()
✖ tsibble::interval()  masks lubridate::interval()
✖ dplyr::lag()         masks stats::lag()
✖ tsibble::setdiff()   masks base::setdiff()
✖ tsibble::union()     masks base::union()



## Prepare data

In [2]:
df_final_train_features = read.csv("df_final_train_features.csv")
df_final_train_features = df_final_train_features %>%
    mutate(cluster = as.factor(cluster)) %>%
    mutate(snap_TX = as.factor(snap_TX))
df_final_test_features = read.csv("df_final_test_features.csv")
df_final_test_features = df_final_test_features %>%
    mutate(cluster= as.factor(cluster)) %>%
    mutate(snap_TX = as.factor(snap_TX))

# create quarter
df_final_train_features = df_final_train_features %>% mutate(quarter = quarter(date))
df_final_test_features = df_final_test_features %>% mutate(quarter = quarter(date))

### Dummy varaibles
df_final_train_features <- dummy_cols(df_final_train_features, select_columns = 'wday', remove_first_dummy= TRUE)
df_final_test_features <- dummy_cols(df_final_test_features, select_columns = 'wday', remove_first_dummy= TRUE)
df_final_train_features <- dummy_cols(df_final_train_features, select_columns = 'quarter', remove_first_dummy= TRUE)
df_final_test_features = df_final_test_features %>%
                            mutate(quarter_2 = 1) %>%
                            mutate(quarter_3 = 0) %>%
                            mutate(quarter_4 = 0) 
df_final_train_features <- dummy_cols(df_final_train_features, select_columns = 'month', remove_first_dummy= TRUE)
df_final_test_features <- df_final_test_features %>%
                            mutate(month_2 = 0) %>%
                            mutate(month_3 = 0) %>%
                            mutate(month_4 = ifelse(month(date) == 4, 1, 0)) %>%
                            mutate(month_5 = ifelse(month(date) == 5, 1, 0)) %>%
                            mutate(month_6 = 0) %>%
                            mutate(month_7 = 0) %>%
                            mutate(month_8 = 0) %>%
                            mutate(month_9 = 0) %>%
                            mutate(month_10 = 0) %>%
                            mutate(month_11 = 0) %>%
                            mutate(month_12 = 0)


ts_final_train_features = df_final_train_features %>%
    mutate(date = as.Date(date)) %>%
    as_tsibble(index=date, key=c("id"))


ts_final_test_features = df_final_test_features %>%
    mutate(date = as.Date(date)) %>%
    as_tsibble(index=date, key=c("id"))


In [3]:
head(ts_final_train_features)
head(ts_final_test_features)

id,wm_yr_wk,date,sales,weekday,wday,month,year,event_name_1,event_type_1,⋯,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
<chr>,<int>,<date>,<int>,<chr>,<int>,<int>,<int>,<chr>,<chr>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
FOODS_3_001,11101,2011-01-29,0,Saturday,1,1,2011,NA,NA,⋯,0,0,0,0,0,0,0,0,0,0
FOODS_3_001,11101,2011-01-30,2,Sunday,2,1,2011,NA,NA,⋯,0,0,0,0,0,0,0,0,0,0
FOODS_3_001,11101,2011-01-31,1,Monday,3,1,2011,NA,NA,⋯,0,0,0,0,0,0,0,0,0,0
FOODS_3_001,11101,2011-02-01,3,Tuesday,4,2,2011,NA,NA,⋯,0,0,0,0,0,0,0,0,0,0
FOODS_3_001,11101,2011-02-02,0,Wednesday,5,2,2011,NA,NA,⋯,0,0,0,0,0,0,0,0,0,0
FOODS_3_001,11101,2011-02-03,0,Thursday,6,2,2011,NA,NA,⋯,0,0,0,0,0,0,0,0,0,0


id,date,sales,wday,month,month_day,year_day,weekend,has_event,has_two_events,⋯,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
<chr>,<date>,<int>,<int>,<int>,<int>,<int>,<lgl>,<lgl>,<lgl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
FOODS_3_001,2016-04-25,1,3,4,25,116,FALSE,FALSE,FALSE,⋯,0,1,0,0,0,0,0,0,0,0
FOODS_3_001,2016-04-26,0,4,4,26,117,FALSE,FALSE,FALSE,⋯,0,1,0,0,0,0,0,0,0,0
FOODS_3_001,2016-04-27,0,5,4,27,118,FALSE,FALSE,FALSE,⋯,0,1,0,0,0,0,0,0,0,0
FOODS_3_001,2016-04-28,1,6,4,28,119,FALSE,FALSE,FALSE,⋯,0,1,0,0,0,0,0,0,0,0
FOODS_3_001,2016-04-29,0,7,4,29,120,FALSE,FALSE,FALSE,⋯,0,1,0,0,0,0,0,0,0,0
FOODS_3_001,2016-04-30,1,1,4,30,121,TRUE,TRUE,FALSE,⋯,0,1,0,0,0,0,0,0,0,0


## HC Clustering Experiments

In [4]:
df_final_train_features %>%
   group_by(cluster)  %>%
   select(cluster,id) %>% 
   unique() %>% 
   summarize(n = n())

cluster,n
<fct>,<int>
1,135
2,121
3,277
4,262
5,11
6,17


In [5]:
num_samples_per_cluster = 5

# select two samples from each cluster
df_final_train_features_mix = df_final_train_features %>%
    group_by(cluster)  %>%
    select(cluster,id) %>% 
    unique() %>% 
    arrange(cluster) %>% 
    sample_n(num_samples_per_cluster) 

### General Model Training based on aggregated results

In [6]:
plan("future::multisession") 

# train on selected samples 2x6
df_final_train_features_aggregated = df_final_train_features %>%
    filter(id %in% df_final_train_features_mix$id) %>%
    group_by(date) %>%
    summarize(sales = mean(sales), 
              weekend = first(weekend), 
              has_event_near = first(has_event_near), 
              has_two_events = first(has_two_events), 
              snap_TX=first(snap_TX),
              month_2=first(month_2),
              month_3=first(month_3),
              month_4=first(month_4),
              month_5=first(month_5),
              month_6=first(month_6),
              month_7=first(month_7),
              month_8=first(month_8),
              month_9=first(month_9),
              month_10=first(month_10),
              month_11=first(month_11),
              month_12=first(month_12),
             )

ts_final_train_features_aggregated = df_final_train_features_aggregated %>%
    mutate(date = as.Date(date)) %>%
    as_tsibble(index=date)

arima_fit <- ts_final_train_features_aggregated %>%
  model(
    auto = ARIMA(sales ~ fourier(K=1) + has_event_near + has_two_events + snap_TX + month_2 + month_3 + month_4 + month_5 + month_6 + month_7+ month_8+ month_9+ month_10+ month_11 + month_12, stepwise = TRUE, approx = TRUE)
  )

### Test for individual points by using the general model

In [7]:
plan("future::multisession") 

df_final_test_features_aggregated = df_final_test_features %>%
    filter(id %in% df_final_train_features_mix$id) %>%
    group_by(date, id) %>%
    summarize(sales = mean(sales), 
              weekend = first(weekend), 
              has_event_near = first(has_event_near), 
              has_two_events = first(has_two_events), 
              snap_TX=first(snap_TX),
              month_2=first(month_2),
              month_3=first(month_3),
              month_4=first(month_4),
              month_5=first(month_5),
              month_6=first(month_6),
              month_7=first(month_7),
              month_8=first(month_8),
              month_9=first(month_9),
              month_10=first(month_10),
              month_11=first(month_11),
              month_12=first(month_12),
              .groups = "drop"
             )

ts_final_test_features_aggregated = df_final_test_features_aggregated %>%
        mutate(date = as.Date(date)) %>%
        as_tsibble(index=date, key=c("id"))

accuracy_arima_list = c()
for (product_id in df_final_train_features_mix$id) {
    
    ts_final_test_features_aggregated_product = ts_final_test_features_aggregated %>%
        filter(id == product_id) %>%
        select(-id) %>%
        select(date, sales:month_12) %>% ungroup()
    arima_fc = arima_fit %>% forecast(ts_final_test_features_aggregated_product)
    accuracy_arima = accuracy(arima_fc, ts_final_test_features_aggregated_product)
    accuracy_arima_list = append(accuracy_arima_list, accuracy_arima$RMSE)
}
accuracy_arima_list
mean_accuracy_arima_without_hc = accuracy_arima_list %>% mean()
mean_accuracy_arima_without_hc

[1] 2.033080 1.645935 1.538552 1.508182 1.568518 1.551838 1.796891 2.978196
 [9] 2.398700 2.365796 3.835955 1.610797 1.779183 1.492412 1.639476 1.718441
[17] 4.830125 1.342477 1.702878 1.961103 3.364364 8.385409 7.199406 8.924314
[25] 7.261749 2.033080 1.984965 3.261125 1.246343 1.612578

[1] 2.885729

### Cluster-based General Model Training

In [8]:
plan("future::multisession") 

df_final_train_features_aggregated = df_final_train_features %>%
    filter(id %in% df_final_train_features_mix$id) %>%
    group_by(date,cluster) %>%
    summarize(sales = mean(sales), weekend = first(weekend), 
              has_event_near = first(has_event_near), 
              has_two_events = first(has_two_events), 
              snap_TX=first(snap_TX),
              month_2=first(month_2),
              month_3=first(month_3),
              month_4=first(month_4),
              month_5=first(month_5),
              month_6=first(month_6),
              month_7=first(month_7),
              month_8=first(month_8),
              month_9=first(month_9),
              month_10=first(month_10),
              month_11=first(month_11),
              month_12=first(month_12),
              .groups="drop"
             )

ts_final_train_features_aggregated = df_final_train_features_aggregated %>%
    mutate(date = as.Date(date)) %>%
    as_tsibble(index=date, key=c("cluster"))

arima_fit <- ts_final_train_features_aggregated %>%
  model(
    auto = ARIMA(sales ~ fourier(K=1) + has_event_near + has_two_events + snap_TX + month_2 + month_3 + month_4 + month_5 + month_6 + month_7+ month_8+ month_9+ month_10+ month_11 + month_12, stepwise = TRUE, approx = TRUE)
  )

### Test for individual points by using the cluster-based general model

In [9]:
plan("future::multisession") 


df_final_test_features_aggregated = df_final_test_features %>%
    filter(id %in% df_final_train_features_mix$id) %>%
    group_by(date, cluster, id) %>%
    summarize(sales = mean(sales),
              weekend = first(weekend),
              has_event_near = first(has_event_near),
              has_two_events = first(has_two_events),
              snap_TX=first(snap_TX),
              month_2=first(month_2),
              month_3=first(month_3),
              month_4=first(month_4),
              month_5=first(month_5),
              month_6=first(month_6),
              month_7=first(month_7),
              month_8=first(month_8),
              month_9=first(month_9),
              month_10=first(month_10),
              month_11=first(month_11),
              month_12=first(month_12),
              .groups = "drop"
             )

ts_final_test_features_aggregated = df_final_test_features_aggregated %>%
        mutate(date = as.Date(date)) %>%
        as_tsibble(index=date, key=c("cluster","id"))

accuracy_arima_list = c()

for (product_id in df_final_train_features_mix$id) {
    ts_final_test_features_aggregated_product = ts_final_test_features_aggregated %>%
        filter(id == product_id) %>%
        select(-id) %>%
        select(date, cluster, sales:month_12) %>% ungroup()
    cluster_num = unique(ts_final_test_features_aggregated_product$cluster)
    arima_fc = arima_fit %>% 
                filter(cluster == cluster_num) %>% 
                forecast(ts_final_test_features_aggregated_product)
    accuracy_arima = accuracy(arima_fc, ts_final_test_features_aggregated_product)
    accuracy_arima_list = append(accuracy_arima_list, accuracy_arima$RMSE)
}

accuracy_arima_list
mean_accuracy_arima_hc = accuracy_arima_list %>% mean()
mean_accuracy_arima_hc

[1] 0.4456258 0.7323827 1.0746505 1.0665621 1.2289204 0.8134519 2.2541373
 [8] 3.4039442 2.7885603 2.3063527 4.2997819 1.2965985 1.1639940 1.4108755
[15] 1.3968815 1.5238348 5.0152948 1.1693128 1.4621370 1.7337189 6.3258581
[22] 8.4194954 5.8354391 6.7302641 6.6234933 0.6520268 0.6347379 3.7599086
[29] 1.2868923 0.6697617

[1] 2.584163

## GMM Clustering Experiments

In [10]:
df_final_train_features %>%
   group_by(cluster_gmm)  %>%
   select(cluster_gmm,id) %>% 
   unique() %>% 
   summarize(n = n())

cluster_gmm,n
<int>,<int>
1,510
2,52
3,126
4,87
5,48


In [11]:
num_samples_per_cluster = 5

df_final_train_features_mix = df_final_train_features %>%
    group_by(cluster_gmm)  %>%
    select(cluster_gmm,id) %>% 
    unique() %>% 
    arrange(cluster_gmm) %>% 
    sample_n(num_samples_per_cluster) 


### General Model Training

In [ ]:
plan("future::multisession") 

# train on selected samples 2x6
df_final_train_features_aggregated = df_final_train_features %>%
    filter(id %in% df_final_train_features_mix$id) %>%
    group_by(date) %>%
    summarize(sales = mean(sales), 
              weekend = first(weekend), 
              has_event_near = first(has_event_near), 
              has_two_events = first(has_two_events), 
              snap_TX=first(snap_TX),
              month_2=first(month_2),
              month_3=first(month_3),
              month_4=first(month_4),
              month_5=first(month_5),
              month_6=first(month_6),
              month_7=first(month_7),
              month_8=first(month_8),
              month_9=first(month_9),
              month_10=first(month_10),
              month_11=first(month_11),
              month_12=first(month_12),
             )

ts_final_train_features_aggregated = df_final_train_features_aggregated %>%
    mutate(date = as.Date(date)) %>%
    as_tsibble(index=date)

arima_fit <- ts_final_train_features_aggregated %>%
  model(
    auto = ARIMA(sales ~ fourier(K=1) + has_event_near + has_two_events + snap_TX + month_2 + month_3 + month_4 + month_5 + month_6 + month_7+ month_8+ month_9+ month_10+ month_11 + month_12, stepwise = TRUE, approx = TRUE)
  )

### Test for individual points by using the general model

In [ ]:
plan("future::multisession") 

df_final_test_features_aggregated = df_final_test_features %>%
    filter(id %in% df_final_train_features_mix$id) %>%
    group_by(date, id) %>%
    summarize(sales = mean(sales), 
              weekend = first(weekend), 
              has_event_near = first(has_event_near), 
              has_two_events = first(has_two_events), 
              snap_TX=first(snap_TX),
              month_2=first(month_2),
              month_3=first(month_3),
              month_4=first(month_4),
              month_5=first(month_5),
              month_6=first(month_6),
              month_7=first(month_7),
              month_8=first(month_8),
              month_9=first(month_9),
              month_10=first(month_10),
              month_11=first(month_11),
              month_12=first(month_12),
              .groups = "drop"
             )

ts_final_test_features_aggregated = df_final_test_features_aggregated %>%
        mutate(date = as.Date(date)) %>%
        as_tsibble(index=date, key=c("id"))

accuracy_arima_list = c()
for (product_id in df_final_train_features_mix$id) {
    
    ts_final_test_features_aggregated_product = ts_final_test_features_aggregated %>%
        filter(id == product_id) %>%
        select(-id) %>%
        select(date, sales:month_12) %>% ungroup()
    arima_fc = arima_fit %>% forecast(ts_final_test_features_aggregated_product)
    accuracy_arima = accuracy(arima_fc, ts_final_test_features_aggregated_product)
    accuracy_arima_list = append(accuracy_arima_list, accuracy_arima$RMSE)
}
accuracy_arima_list
mean_accuracy_arima_without_gmm = accuracy_arima_list %>% mean()
mean_accuracy_arima_without_gmm

### Cluster-based General Model Training

In [ ]:
plan("future::multisession") 


df_final_train_features_aggregated = df_final_train_features %>%
    filter(id %in% df_final_train_features_mix$id) %>%
    group_by(date,cluster_gmm) %>%
    summarize(sales = mean(sales), weekend = first(weekend), 
              has_event_near = first(has_event_near), 
              has_two_events = first(has_two_events), 
              snap_TX=first(snap_TX),
              month_2=first(month_2),
              month_3=first(month_3),
              month_4=first(month_4),
              month_5=first(month_5),
              month_6=first(month_6),
              month_7=first(month_7),
              month_8=first(month_8),
              month_9=first(month_9),
              month_10=first(month_10),
              month_11=first(month_11),
              month_12=first(month_12),
              .groups="drop"
             )

ts_final_train_features_aggregated = df_final_train_features_aggregated %>%
    mutate(date = as.Date(date)) %>%
    as_tsibble(index=date, key=c("cluster_gmm"))

arima_fit <- ts_final_train_features_aggregated %>%
  model(
    auto = ARIMA(sales ~ fourier(K=1) + has_event_near + has_two_events + snap_TX + month_2 + month_3 + month_4 + month_5 + month_6 + month_7+ month_8+ month_9+ month_10+ month_11 + month_12, stepwise = TRUE, approx = TRUE)
  )

### Test for individual points by using the gmm cluster-based general model

In [ ]:
plan("future::multisession") 


df_final_test_features_aggregated = df_final_test_features %>%
    filter(id %in% df_final_train_features_mix$id) %>%
    group_by(date, cluster_gmm, id) %>%
    summarize(sales = mean(sales),
              weekend = first(weekend),
              has_event_near = first(has_event_near),
              has_two_events = first(has_two_events),
              snap_TX=first(snap_TX),
              month_2=first(month_2),
              month_3=first(month_3),
              month_4=first(month_4),
              month_5=first(month_5),
              month_6=first(month_6),
              month_7=first(month_7),
              month_8=first(month_8),
              month_9=first(month_9),
              month_10=first(month_10),
              month_11=first(month_11),
              month_12=first(month_12),
              .groups = "drop"
             )

ts_final_test_features_aggregated = df_final_test_features_aggregated %>%
        mutate(date = as.Date(date)) %>%
        as_tsibble(index=date, key=c("cluster_gmm","id"))

accuracy_arima_list = c()

for (product_id in df_final_train_features_mix$id) {
    ts_final_test_features_aggregated_product = ts_final_test_features_aggregated %>%
        filter(id == product_id) %>%
        select(-id) %>%
        select(date, cluster_gmm, sales:month_12) %>% ungroup()
    cluster_num = unique(ts_final_test_features_aggregated_product$cluster_gmm)
    arima_fc = arima_fit %>% 
                filter(cluster_gmm == cluster_num) %>% 
                forecast(ts_final_test_features_aggregated_product)
    accuracy_arima = accuracy(arima_fc, ts_final_test_features_aggregated_product)
    accuracy_arima_list = append(accuracy_arima_list, accuracy_arima$RMSE)
}

accuracy_arima_list
mean_accuracy_arima_gmm = accuracy_arima_list %>% mean()
mean_accuracy_arima_gmm

## Results Summary

In [ ]:
mean_accuracy_arima_without_hc
mean_accuracy_arima_hc
mean_accuracy_arima_without_gmm
mean_accuracy_arima_gmm